In [ ]:
import re
import itertools
import pandas as pd
from tqdm import tqdm
import pickle
import json

In [ ]:
folder='dataset_v5/v5_2/'

In [3]:
def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [4]:
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

In [5]:
ment=pd.read_csv(folder+"/processed_sodhganga_mentorship_dept_rev_with_initial_ids.csv",sep=",")

In [6]:
ment.shape

(205744, 25)

In [7]:
#sim_names1 = load_obj("index_files4/0_100000.pkl")

In [8]:
sim_names1 = load_obj(folder+"sn_0_50000.pkl")
sim_names2 = load_obj(folder+"sn_49900_100000.pkl")
sim_names3 = load_obj(folder+"sn_99900_200000.pkl")
sim_names4 = load_obj(folder+"sn_199900_281655.pkl")

In [9]:
sim_names5 = load_obj(folder+"sim_name_in_operator.pkl")

In [10]:
#sim_names1+sim_names5

In [11]:
sim_names5[0]

('Goswami, Chandan', 86803, 'Goswami, Chandana', 86804)

In [ ]:
#sim_names2 = load_obj("index_files4/100000_200000.pkl")

In [ ]:
#sim_names3 = load_obj("index_files4/200000_281425.pkl")

In [ ]:
#sim_names3

In [ ]:
#tmp1=Merge(sim_names1, sim_names2)

In [ ]:
#final_names = Merge(tmp1, sim_names3)

In [12]:
final_names=sim_names1 + sim_names2 + sim_names3 + sim_names4

In [13]:
final_names1=final_names+sim_names5

In [14]:
len(final_names)

27958

In [15]:
below_ten=[name for name in final_names if max(name[4:8]) <= 10]

In [ ]:
#below_ten

In [ ]:
#list(sim_names1.keys())[-5:]

In [ ]:
#list(sim_names2.keys())[0:5]

In [ ]:
#ment['advisorId']

In [17]:
len(final_names1)

29339

In [ ]:
#final_names[-10:]

In [18]:
ment = ment.sort_values(by=['advisorId','researcherId'])

In [ ]:
# sim_dict={}
# for key in final_names:
#     sim_dict[(final_names[key][1],final_names[key][3])]=(final_names[key][0],final_names[key][2])

In [ ]:
sim_dict=[]
for key in final_names1:
    tmp=ment[(ment['advisorId']==key[1])|(ment['researcherId']==key[1])]
    inst = list(pd.unique(tmp['publisher_institution']))
    dept = list(pd.unique(tmp['publisher_dept']))
    sim_dict.append([key[0],key[1],inst,dept])
    tmp=ment[(ment['advisorId']==key[3])|(ment['researcherId']==key[3])]
    inst = list(pd.unique(tmp['publisher_institution']))
    dept = list(pd.unique(tmp['publisher_dept']))
    sim_dict.append([key[2],key[3],inst,dept])

In [ ]:
sim_df=pd.DataFrame(sim_dict)

In [ ]:
sim_df.to_csv(folder+"/disambiguated_names.csv",sep=",",index=False)

In [ ]:
#sim_df.iloc[0:100].values

In [ ]:
#sim_df.iloc[-100:].values

In [19]:
sim_dict1={}
for key in final_names1:
    if key[1]<key[3]:
        sim_dict1[(key[1],key[3])]=(key[0],key[2])
    else:
        sim_dict1[(key[3],key[1])]=(key[2],key[0])

In [20]:
len(sim_dict1)

28386

In [21]:
sorted_sim_dict=sorted(sim_dict1.items())

In [ ]:
tuple(sorted((10,8)))

In [ ]:
sorted_sim_dict[-5:]

In [22]:
len(set(sorted_sim_dict))

28386

In [ ]:
count=1
for a in sorted_sim_dict:
    if (a[1][0] in a[1][1]) or (a[1][1] in a[1][0]):
        count+=1
        print(a)

In [ ]:
#ment[ment['researcherId']==201704]

In [ ]:
((23082, 23433), ('Chatterjee, S. N.', 'Chatterjee, S.'))
((23111, 23433), ('Chatterjee, S. D.', 'Chatterjee, S.'))
((23353, 23409), ('Bhattacharya, S.', 'Bhattacharya, S. K.'))
((23479, 26353), ('Bhattacharyya, S. P.', 'Bhattacharyya, S.'))
((28450, 28466), ('Shanmugasundaram', 'Shanmugasundaramerb'))
((28450, 30805), ('Shanmugasundaram', 'Shanmugasundaram E. R. S.'))
((28462, 28472), ('Shanmugasundramerb', 'Shanmugasundram'))
((37007, 37010), ('Chatterjee, N. C.', 'Chatterjee, N. C'))
((37007, 37049), ('Chatterjee, N. C.', 'Chatterjee'))
((37010, 37049), ('Chatterjee, N. C', 'Chatterjee'))
((37013, 37049), ('Chatterjee, S. K.', 'Chatterjee'))
((37032, 37049), ('Chatterjee, S. P.', 'Chatterjee'))
((50010, 190725), ('Elangovan', 'Elangovan, P.'))
((50010, 190784), ('Elangovan', 'Elangovan, M.'))
((50010, 190893), ('Elangovan', 'Elangovan, R.'))

In [ ]:
count

In [ ]:
for a in sorted_sim_dict:
    if a[0][0]==85852 or a[0][1]==85852	:
        print(a)

In [ ]:
#sorted_sim_dict[-1000:]

In [23]:
ment['advId'] = ment['advisorId']
ment['resId'] = ment['researcherId']

In [24]:
tmp_dict={}
for a in sorted_sim_dict:
    if a[0][0] in tmp_dict and a[0][1] in tmp_dict:
        ment.loc[ment['advId'].isin([tmp_dict[a[0][0]],tmp_dict[a[0][1]]]),'advId']= tmp_dict[a[0][0]]
        ment.loc[ment['resId'].isin([tmp_dict[a[0][0]],tmp_dict[a[0][1]]]),'resId']= tmp_dict[a[0][0]]
    elif a[0][0] in tmp_dict:
        ment.loc[ment['advId'].isin([tmp_dict[a[0][0]],a[0][1]]),'advId']= tmp_dict[a[0][0]]
        ment.loc[ment['resId'].isin([tmp_dict[a[0][0]],a[0][1]]),'resId']= tmp_dict[a[0][0]]
    elif a[0][1] in tmp_dict:
        ment.loc[ment['advId'].isin([a[0][0],tmp_dict[a[0][1]]]),'advId']= a[0][0]
        ment.loc[ment['resId'].isin([a[0][0],tmp_dict[a[0][1]]]),'resId']= a[0][0]
    else:
        tmp_dict[a[0][1]]=a[0][0]
        ment.loc[ment['advId'].isin([a[0][0],a[0][1]]),'advId']= a[0][0]
        ment.loc[ment['resId'].isin([a[0][0],a[0][1]]),'resId']= a[0][0]

In [ ]:
#tmp_dict

In [ ]:
# nelg=0
# for a in sorted_sim_dict:
#     if a[0][0] in ment['resId'].values and a[0][1] in ment['resId'].values:
#         nelg+=1
#     else:
#         ment.loc[ment['advId'].isin(a[0]),'advId']= max(a[0])
#         ment.loc[ment['resId'].isin(a[0]),'resId']= max(a[0])

In [ ]:
#folder+"shodhganga_ment_with_disamb_ids.csv"

In [25]:
ment.to_csv(folder+"shodhganga_ment_with_disamb_ids.csv",sep=",", index=False)

In [26]:
pd.unique(ment['publisher_dept']).shape

(2825,)

In [27]:
pd.unique(ment['DepartmentId']).shape

(1440,)

In [28]:
pd.unique(ment['researcherId']).shape

(191766,)

In [29]:
pd.unique(ment['resId']).shape

(188271,)

In [30]:
pd.unique(ment['advisorId']).shape

(93541,)

In [31]:
pd.unique(ment['advId']).shape

(76840,)

In [32]:
ment.columns

Index(['advisorId', 'researcherId', 'advisor_name', 'researcher_name',
       'dc.language.iso[]', 'publisher_dept', 'DepartmentId',
       'publisher_institution', 'instituteId', 'title', 'thesisId',
       'dc.type.degree', 'date_submitted', 'dc.date.awarded', 'dc.subject.ddc',
       'dc.description.abstract', 'dc.title.alternative',
       'uniq_ddc_across_dept', 'N_DepartmentId_1', 'N_DepartmentId',
       'previous_advisor_name', 'advisor_inst_dept', 'res_inst_dept',
       'adv_names_rid', 'res_names_rid', 'advId', 'resId'],
      dtype='object')

In [35]:
ment[(ment['advisor_name'].str.contains('Ajith')) & (ment['instituteId']=='I251')][['advId','publisher_dept']].value_counts()#['publisher_dept'].values

advId  publisher_dept                                                                          
11     department of sacred heart college                                                          8
28     department of economics                                                                     7
66784  department of n.s.s. hindu college                                                          6
74845  department of hindi                                                                         1
28     department of sacred heart college(thevara).departments of economics and research centre    1
       department of sacred heart college(thevara). department of economics and research centre    1
       department of economics and research centre                                                 1
dtype: int64

In [ ]:
#ment[(ment['advisor_name'].str.contains('Ajith')) & (ment['instituteId']=='I251')][['publisher_dept',"dc.subject.ddc"]].value_counts()

In [ ]:
#ment[(ment['advisor_name'].str.contains('Ajith')) & (ment['instituteId']=='I251')]#[['advisorId','publisher_dept']].value_counts()#['publisher_dept'].values

In [ ]:
#advisor_detail=ment.groupby(['advId','instituteId','DepartmentId'], as_index=False)[['advisor_name','publisher_dept','publisher_institution','date_submitted','title','dc.subject.ddc']].agg(lambda x: list(x))

In [ ]:
#advisor_detail.shape

In [ ]:
ment.columns

In [34]:
ment[ment['previous_advisor_name'].str.contains("V. Rama Mohan Gupta, N. Devanna")]

,advisorId,researcherId,advisor_name,researcher_name,dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,title,...,uniq_ddc_across_dept,N_DepartmentId_1,N_DepartmentId,previous_advisor_name,advisor_inst_dept,res_inst_dept,adv_names_rid,res_names_rid,advId,resId
201030,91434,277338,"V. Rama Mohan Gupta, N. Devanna","Srikanth, K.",eng,department of pharmaceutical sciences,D1978,"Jawaharlal Nehru Technological University, Ana...",I346,Selection of the best vesicular carrier system...,...,"('600', '610', '615')",D1978,D1978,"V. Rama Mohan Gupta, N. Devanna","V. Rama Mohan Gupta, N. Devanna@I346@D1978","Srikanth, K.@I346@D1978","V. Rama Mohan Gupta, N. Devanna@91434","Srikanth, K.@277338",91434,277338


In [ ]:
#ment[ment['advId']==37049]

In [ ]:
ment[ment['advisorId']==11864]

In [ ]:
ment[ment['advisorId']==11871]